In [1]:
from translate import Translator
import cobra
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
model_fn = 'models/b_licheniformis/1-s2.0-S0923250816000024-mmc1.xml'

In [ ]:
model = cobra.io.read_sbml_model(model_fn)

In [ ]:
def bacillus_subtilis():
    # Load original model
    name = "Bacillus subtilis"
    model_info = get_model_info(name)
    logging.info(model_info)

    # Load model
    model_path = get_original_models_folder() / "{0}.xml".format(model_info["Model"])
    model = cobra.io.read_sbml_model(str(model_path))

    # Change default bounds
    for r in model.reactions:
        if r.lower_bound < -1000:
            r.lower_bound = -1000
        if r.upper_bound > 1000:
            r.upper_bound = 1000

    # Remove universally blocked reactions
    blocked_reactions = cobra.flux_analysis.find_blocked_reactions(model, open_exchanges = True)
    model.remove_reactions(blocked_reactions, remove_orphans = True)
    print("Removed {0} blocked reactions".format(len(blocked_reactions)))

    # The iBsu1103 model has a strange implementation of exchange reactions with "boundary" metabolites, appended "_b" in the id. These can be deleted
    n_b = 0
    mets_to_remove = []
    for m in model.metabolites:
        if m.id[-2:] =="_b":
            mets_to_remove.append(m)
            n_b += 1

    model.remove_metabolites(mets_to_remove)
    logging.info("Deleted {0} boundary metabolites from iBsu1103".format(n_b))

    n_r = 0
    rxns_to_remove = []
    for r in model.reactions:
        if len(r.metabolites) == 0:
            rxns_to_remove.append(r)
            n_r += 1
    
    model.remove_reactions(rxns_to_remove, remove_orphans = True)
    logging.info("Deleted {0} boundary reactions from iBsu1103".format(n_b))
    model.repair()
    model.optimize()

    if not _has_curated_mapping(name):
        # Get the translator
        T = Translator()

        # # Map extracellular metabolites to BiGG
        bigg_match_list = []
        for m in model.metabolites:
            if m.compartment == "e":
                seed_id = m.id.split("_")[0]
                matched_IDs = T.translate_met(seed_id, "seed", "bigg", return_all = True)
                bigg_match_list.append([m.id, m.name, seed_id, "; ".join(matched_IDs)])

        # Store draft metabolite curations as csv
        _store_draft_met_id_translation(name, bigg_match_list, col3 = "seed ID")

    else:
        # The next steps require that a curated version of the draft metabolite ID mapping is added to the "curated" folder
        # Read curated csv-file
        _convert_met_ids_from_table(name, model)

        # Make common met ID changes
        model = _additional_met_ID_curations(model)

        # Change id of biomass
        r_biomass = model.reactions.get_by_id("bio00127")
        r_biomass.id = "Biomass"

        # Add exchange reactions for all extracellular media components
        add_exchange_reactions(model)

        save_curated_model_as_yml(model, model_info)
        save_curated_model_as_xml(model, model_info)